# Lab - 6

Name: Purusharth Malik

Registration No.: 2348542

### Importing the dataset

In [1]:
import pandas as pd
data=pd.read_csv('/content/newdata.csv')
data.head()

,Unnamed: 0,english_sentence,hindi_sentence
0,0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


### Loading the Tokenizer and model

In [11]:
from transformers import MarianTokenizer, TFMarianMTModel
tokenizer=MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model=TFMarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

### Preprocessing the data

In [12]:
input=tokenizer(data['english_sentence'].tolist(),truncation=True,padding=True,max_length=128)
target=tokenizer(data['hindi_sentence'].tolist(),truncation=True,padding=True,max_length=128)

### Converting to tensorflow dataset

In [13]:
import tensorflow as tf
input_ids=tf.convert_to_tensor(input['input_ids'])
attention_masks=tf.convert_to_tensor(input['attention_mask'])
decoder_input_ids=tf.convert_to_tensor(target['input_ids'])

### Shifting decoder_input_ids by one position to the right for teacher forcing

In [14]:
labels=tf.roll(decoder_input_ids,shift=-1,axis=1)
labels=tf.where(labels==0,-100,labels)

### Creating a tensorflow dataset

In [24]:
dataset=tf.data.Dataset.from_tensor_slices(({
    'input_ids':input_ids,
    'attention_mask':attention_masks,
    'decoder_input_ids':decoder_input_ids
},labels))
dataset=dataset.shuffle(1000).batch(16)

### Compiling the model

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss)

### Fitting the model

In [26]:
model.fit(dataset, epochs=1)

1804/1804 [==============================] - 779s 383ms/step - loss: nan


### Translating English to Hindi

In [2]:
text="Himanshu is a good boy"
input_ids = tokenizer.encode(text, return_tensors="tf")
generated_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
translated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Translated Hindi text: ", translated_text)

Translated Hindi text: हिमांशु एक अच्छा लड़का है


# End